In [28]:
import igraph as ig
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import ast
from src.utils import tweets_as_dataframe

In [ ]:
raw_data = tweets_as_dataframe("../../../data/tweets.dat")

In [ ]:
raw_data.info()

In [44]:
def parse_referenced_tweets(row: pd.Series):
    action = row["referenced_tweets"]
    if isinstance(row, float):
        return None
    try:
        if isinstance(row, list) and parsed_list:
            nested_df = pd.DataFrame(parsed_list)
            print(nested_df)
            return nested_df
    except (ValueError, SyntaxError):
        print(f"Warning: Failed to parse {row}")
        return None
set(raw_data.apply(parse_referenced_tweets, axis=1))

{None}

In [26]:
graph = ig.Graph()
raw_data.loc[5]["referenced_tweets"]
# raw_data[raw_data["id"] == "675768250114940928"]

[{'type': 'retweeted', 'id': '675805684034859010'}]